## Key Value Store
A non-relational database which supports two operations - `get(key)` and `put(key, value)` to retrieve and store key-value pairs. Some examples include Redis, Memcached, Amazon Dynamo, etc.

Key needs to be hashable and the value is any object. Some design considerations:
- should be scalable
- choice between consistency and availability (tunable consistency)
- low latency

## Single Server Store
Implementation of a single server store is relatively simple - use a in-memory hashtable. However, memory is limited and we'll soon fill it up. One approach to mitigate the problem is to have an LRU cache and evict key-value pair to disk. Additionally, we can try compressing values.

If a key-value pair is not stored in disk, the `get` operation becomes slow since it has to fetch data from disk. To support large amount of data, we need to shift to using a distributed approach.

## Distributed Key-Value Store
Key-value pairs are distributed over multiple machines. In a distributed system, CAP theorem applies, therefore we can guarantee either consistency or availability. As an example, let's consider a distributed system consisting of 3 nodes  $n_1$, $n_2$ and $n_3$. And $n_3$ goes down.  
**Consistency:** if we chooose consistency, we block read and writes from $n_1$ and $n_2$ till $n_3$ is back online. The service returns error.  
**Availability:** if we choose availability, the system keeps accepting reads, even though it might return stale data. For writes, $n_1$ and $n_2$ will keep accepting writes, and data will be synced to $n_3$ when the network partition is resolved.

### Partitioning
One server can't possibly store all the data, so we split data into smaller partitions and each node stores a fraction of the total data. Consistent hasing allows us to:
- spread data evenly across multiple nodes
- minimise data movement when nodes are added/removed.

<img src="images/key_value_consistent_hash.png" />

Any of the nodes can receive request from the client. The node receiving the request is referred to as coordinator. The coordinator calculates hash and then forwards the request to the right node.

The number of virtual nodes for a server is proportional to the server capacity. For example, servers with higher capacity are assigned with more virtual nodes.

### Replication
To achieve high availability and reliability, data must be replicated asynchronously over $N$ servers, where $N$ is a configurable parameter called as *replication factor*. For example, if replication factor is set to 3, here is how a `put` operation works:  

<img src="images/key_value_replication.png"/>

For better reliability, replicas are placed in distinct data centers, and data centers are connected through high-speed networks.

### Consistency
Let there be a distributed system with replication factor $N$. For a write operation to be successful, it must be acknowledged by $W$ number of replicas. Similarly, for a read operation to be successful, it must be acknowledged by $R$ number of replicas.  

As an example, setting $W=1$ means we get fast writes since acknowledgement from just one node is enough. The system also has high availability (at the cost of consistency). Configuration of $W$, $R$ and $N$ is a typical tradeoff between latency and consistency. Some configurations:
- $R= 1$ and $W=N$, the system is optimized for a fast read.
- $W=1$ and $R=N$, the system is optimized for fast write.
- $W+R>N$, strong consistency is guaranteed (Usually $N=3, W=R=2$).
- $W+R<=N$, strong consistency is not guaranteed.

**Strong Consistency:** to ensure strong consistency, $W=N$. Additionally, concurrent writes for the same key should not be allowed.
**Weak consistency:** subsequent read operations may not see the most updated value.

<img src="images/key_value_inconsistent.png" />

**Eventual consistency:** this is a specific form of weak consistency. Given enough time, all updates are propagated, and all replicas are consistent.

### Inconsistency Resolution
One way to resolve inconsistency is to maintain a timestamp alongwith every write operation. On a single machine, maintaining sequence of events is easy - maintain timestamp of each event. The same physical clock is used for all timestamps providing a global view of time.

When we move to distributed system, we cannot reply on this property anymore. We cannot guarantee that each machine's clock has the same exact time. Therefore relying on the last timestamp to resolve inconsistency (last write wins), is not the best approach.

**Vector Clock** [explanation](https://riak.com/posts/technical/vector-clocks-revisited/index.html?p=9545.html)

### Failure Detection
To know which nodes are UP, every node sends heartbeats to every other node in the system. However, this is inefficient when many servers are in the system - requires $O(n^2)$ number of heartbeats.

**Gossip Protocol:** is a decentralized peer-to-peer communication technique to transmit messages in an enormous distributed system. Gossip protocol is simple in concept. Each node sends out some data to a set of other nodes. Data propagates through the system node by node like a virus. Eventually data propagates to every node in the system. It's a way for nodes to build a global map from limited local interactions. Below is a sample implementation:

In [ ]:
// Class representing information about a node
class NodeInfo {
    long id;
    int lastHeartbeat;
    long lastTimestamp;
    volatile boolean isAlive = true;
    
    public NodeInfo(long id, int lastHeartbeat, long lastTimestamp) {
        this.id = id;
        this.lastHeartbeat = lastHeartbeat;
        this.lastTimestamp = lastTimestamp;
    }
    
    public void synchronized incrementHeartbeat() {
        this.lastHeartbeat++;
        this.lastTimestamp = System.currentTimeMillis();
    }
    
    public void synchronized copy(NodeInfo nodeInfo) {
        if(nodeInfo != null) {
            // Stale NodeInfo
            if(nodeInfo.lastTimestamp <= this.lastTimestamp) return;
            if(nodeInfo.lastHeartbeat <= this.lastHeartbeat) return;
            
            this.lastTimestamp = nodeInfo.lastTimestamp;
            this.lastHeartbeat = nodeInfo.lastHeartbeat;
            this.isAlive = true;
        }    
    }
    
    public void setIsAlive(boolean value) {
        this.isAlive = value;
    }
}

// Every node also maintains a list of NodeInfo objects
class NodeInfoMap {
    Map<Long, NodeInfo> nodeInfoMap = new HashMap<>();
    final long ID;
    
    // Initialise a node with other seed nodes
    public NodeInfoMap(long id, List<NodeInfo> nodeInfos) {
        this.ID = id;
        for(NodeInfo nodeInfo: nodeInfos) {
            nodeInfo.put(nodeInfo.id, nodeInfo);
        }
    }
    
    public synchronized void updateAll(List<NodeInfo> nodeInfoList) {
        if(nodeInfoList != null) {
            for(NodeInfo n: nodeInfoList) {
                NodeInfo nodeInfo = new NodeInfo(n.id, n.lastHeartbeat, n.lastTimestamp);
                if(nodeInfoMap.contains(nodeInfo.id)) {
                    NodeInfo existingNodeInfo = nodeInfoMap.get(nodeInfo.id);
                    existingNodeInfo.copy(nodeInfo);
                } else {
                    nodeInfoMap.put(nodeInfo.id, nodeInfo);
                }
            }
        }
    }
    
    public synchronized void updateSelf() {
        nodeInfoMap.get(ID).incrementHeartbeat();
    }
}

Every node also supports the following two REST operations: i) GET information about all nodes ii) POST operation that updates all node information sent by another node (through heartbeat). There is also a timer task executed by a node which:
- increments its own heartbeat counter
- moves the peers it hasn’t had a heartbeat exceeding the threshold to “suspected” state
- removes the peers in suspected state which haven’t had any heartbeats exceeding the threshold
- randomly pick $n$ peers to send this node’s membership list to

In [ ]:
class HeartbeatTask extends TimerTask {
    @Override
    public void run() {
        nodeInfoMap.updateSelf();
        
        nodeInfoMap.detectSuspiciousNodes(SUSPICION_THRESHOLD, PROTOCOL_PERIOD);
        nodeInfoMap.removeDeadNodes(FAILURE_THRESHOLD, PROTOCOL_PERIOD);
        
        for(NodeInfo nodeInfo: nodeInfoMap.pickRandom(COUNT)) {
            try {
                HttpClient client = HttpClient.newBuilder().build();
                HttpRequest request = HttpRequest.newBuilder()
                    .uri(URI.create(NODE_DOMAIN + ":" + (nodeInfo.id + 8080)))
                    .POST(BodyPublishers.ofString(nodeInfoMap))
                    .build();
                client.send(request, BodyHandlers.discarding());
            } catch (Exception e) {
                // log
            }
        }
    }
}

### Hinted Handoff
When a server goes down, unlike in a strongly consistent system where read and writes are blocked, in an available system, the first $W$ healthy servers for writes and first $R$
healthy servers for reads are chosen on the hash ring. Offline servers are ignored.

If a server is unavailable due to network or server failures, another server will process requests temporarily. When the down server is up, changes will be pushed back to achieve data consistency. This process is called *hinted handoff*. 